# RESNET50 Image Classification Model Acceleration

In [1]:
%pip install torch torchvision 

In [7]:
import torch 
from torchvision.io import decode_image 
from torchvision.models import resnet50, ResNet50_Weights 

In [8]:
#use the best available weights 
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
#set to inference mode, this disables dropouts and batch normalizations
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

weights are related to a certain trainig set, thus inference images should be preprocessed to match the characteristics of the training data.  
it's easy to get the associated preprocessing pipeline in torchvision.   
**preprocessing always happends on CPU**, so no need to specify the device before defining it 

In [9]:
preprocess = weights.transforms()

check whether GPU is available or not: 

In [11]:
print("GPU is available" if torch.cuda.is_available() else "GPU is not available")

GPU is available


In [ ]:
def inference(image, device):
    #move the model to device
    model.to(device)
    x = preprocess(image).unsqueeze(0)
    #move the tensor to device
    x.to(device)

    with torch.no_grad():
        prediction = model(x)

    class_id = prediction.argmax().item()
    label = weights.meta["categories"][class_id]
    score = prediction[class_id].item()
  
    print(f"{label}: {100 * score:.1f}%")